# SQL II: Grouping and Joining

_September 21, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [3]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

In [20]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Raizel","Mitch","Eric"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Mike W","Mitch","Mike R"],
                            "quiz_score":np.random.randint(0,10,3)})

In [21]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [22]:
# insert these datasets into our db
auto.to_sql('auto2', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

/Users/yunghanjeong/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [23]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [24]:
# for the tips df/table, get the total number of man and woman
tips.groupby("sex").sex.count()

sex
Male      157
Female     87
Name: sex, dtype: int64

Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [26]:
# in sql 
query = """SELECT sex, COUNT(sex) FROM tips GROUP BY sex"""
c.execute(query).fetchall()

[('Female', 87), ('Male', 157)]

In [29]:
# using pandas, select the average amount tipped for time
tips.groupby("time").tip.mean()

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [28]:
# use sql to recreate this query
query = """SELECT time, AVG(tip) FROM tips GROUP BY time"""
c.execute(query).fetchall()

[('Dinner', 3.102670454545454), ('Lunch', 2.7280882352941176)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [31]:
# use pandas, get the average total bills for a party larger than 2 for every size of party

# using pandas
tips[tips["size"] > 2].groupby("size").total_bill.mean()

size
3    23.277632
4    28.613514
5    30.068000
6    34.830000
Name: total_bill, dtype: float64

In [34]:
# use sql to recreate this query
query = """SELECT size, AVG(total_bill) FROM tips 
        WHERE size > 2
        GROUP BY size"""
c.execute(query).fetchall()

[(3, 23.27763157894737), (4, 28.61351351351351), (5, 30.068), (6, 34.83)]

In [46]:
# your turn - use pandas, get the maximum value of total bill for female non smoker

# using group by?
nonsmokerfemale = tips[(tips["sex"] == "Female") & (tips["smoker"] == "No")]
nonsmokerfemale.groupby(["sex"]).total_bill.max()["Female"]
#.total_bill.max()

35.83

In [54]:
tips.groupby(["sex", "smoker"]).total_bill.max()["Female"]["No"]

35.83

In [47]:
# how to get this without using group by?
tips[(tips["sex"] == "Female") & (tips["smoker"] == "No")].total_bill.max()

35.83

In [62]:
# use sql to recreate this query
query = """SELECT MAX(total_bill) FROM tips
        WHERE sex = "Female" AND smoker = "No"
        Group By sex
        """
c.execute(query).fetchall()

[(35.83,)]

In [61]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips
tips.groupby("size").total_bill.max().sort_values(ascending = False)[:2]


size
3    50.81
4    48.33
Name: total_bill, dtype: float64

In [ ]:
# use sql 
query = None
c.execute(query).fetchall()

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [63]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0   Mike W           2
1    Mitch           9
2   Mike R           0
  students  projects  grades
0   Raizel         1      97
1    Mitch         2      83
2     Eric         1      83


In [64]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, left_on = 'students', right_on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Mike W,2,NaN,NaN
1,Mitch,9,2.0,83.0
2,Mike R,0,NaN,NaN


In [74]:
# sql 
query = """SELECT * FROM small_grade
        LEFT JOIN small_quiz 
        ON small_grade.students = small_quiz.students
        """
c.execute(query).fetchall()

[(2, 'Mike R', 0, None, None, None, None),
 (0, 'Mike W', 2, None, None, None, None),
 (1, 'Mitch', 9, 1, 'Mitch', 2, 83)]

In [75]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

,students,quiz_score,projects,grades
0,Mitch,9,2,83


In [76]:
query = """SELECT * FROM small_quiz JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(1, 'Mitch', 2, 83, 1, 'Mitch', 9)]

In [ ]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [79]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` 
        WHERE left_table.id not in 
        (SELECT id FROM right_table) """

OperationalError: no such table: left table

In [80]:
# get median - use offset 
query = """SELECT col FROM table 
        ORDER BY col LIMIT 1 
        OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""

OperationalError: near "table": syntax error